# Практическая работа №4
### Выполнил студент группы БВТ2102 Маширин Федор Сергеевич

Вариант 4
(a or b) and (b or c)

#### Цель работы:
Необходимо реализовать нейронную сеть вычисляющую результат заданной логической операции. Затем реализовать функции, которые будут симулировать работу построенной модели. Функции должны принимать тензор входных данных и список весов.<br>
#### Задание:
Должно быть реализовано 2 функции:
1. Функция, в которой все операции реализованы как поэлементные операции над тензорами.
2. Функция, в которой все операции реализованы с использованием операций над тензорами из NumPy.

Для проверки корректности работы функций необходимо:
1. Инициализировать модель и получить из нее веса.
2. Прогнать датасет через не обученную модель и реализованные 2 функции. Сравнить результат.
3. Обучить модель и получить веса после обучения
4. Прогнать датасет через обученную модель и реализованные 2 функции. Сравнить результат.

*Примечание: так как множество всех наблюдений ограничено, то обучение проводить можно на всем датасете без контроля.*



In [1]:
import numpy as np
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

In [6]:
# Определение логической операции (a or b) and (b or c)
def logical_operation(a, b, c):
  return (a or b) and (b or c)

data = [(a, b, c, logical_operation(a, b, c)) for a in [0, 1] for b in [0, 1] for c in [0, 1]]
X = np.array([[a, b, c] for a, b, c, _ in data], dtype=np.float32)
y = np.array([[out] for _, _, _, out in data], dtype=np.float32)

In [7]:
for x in zip(X, y):
    print(*x)

[0. 0. 0.] [0.]
[0. 0. 1.] [0.]
[0. 1. 0.] [1.]
[0. 1. 1.] [1.]
[1. 0. 0.] [0.]
[1. 0. 1.] [1.]
[1. 1. 0.] [1.]
[1. 1. 1.] [1.]


In [37]:
# Создание модели
model = Sequential()
model.add(Dense(3, input_shape=(3,), activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# Получение начальных весов
initial_weights = model.get_weights()
initial_weights

[array([[ 0.22698689,  0.08038545, -0.2881286 ],
        [-0.26394892, -0.58851314, -0.59928703],
        [ 0.8222959 , -0.7742624 ,  0.67698765]], dtype=float32),
 array([0., 0., 0.], dtype=float32),
 array([[0.07938039],
        [0.5206938 ],
        [0.5582528 ]], dtype=float32),
 array([0.], dtype=float32)]

In [9]:
def exp(x):
  return 2.718281828459045 ** x

In [10]:
# Функция, в которой все операции реализованы как поэлементные операции над тензорами
def elem_tensor_function(inputs, weights):
  W1, b1, W2, b2 = weights

  # Инициализация скрытого слоя
  H = []
  for i in range(len(inputs)):
    h = []
    for j in range(len(W1)):
      sum_val = 0
      for k in range(len(W1[j])):
        sum_val += inputs[i][k] * W1[j][k]
      sum_val += b1[j]
      h.append(max(0, sum_val))
    H.append(h)

  # Инициализация выходного слоя
  Y_pred = []
  for i in range(len(H)):
    sum_val = 0
    for j in range(len(W2)):
      sum_val += H[i][j] * W2[j]
    sum_val += b2[0]
    Y_pred.append(1 / (1 + exp(-sum_val)))

  return np.array(Y_pred)


In [11]:
# Функция, в которой все операции реализованы с использованием операций над тензорами из NumPy
def numpy_tensor_function(inputs, weights):
  W1, b1, W2, b2 = weights

  H = np.maximum(0, np.dot(inputs, W1) + b1)  # Скрытый слой
  Y_pred = 1 / (1 + np.exp(-np.dot(H, W2) - b2))  # Выходной слой

  return Y_pred

In [25]:
# Сравнение результатов на не обученной модели
print("Сравнение на не обученной модели:")
model_untrained_preds = model.predict(X)
elem_tensor_preds = elem_tensor_function(X, initial_weights)
numpy_tensor_preds = numpy_tensor_function(X, initial_weights)
print("Модель:", model_untrained_preds.flatten())
print("Поэлементная функция тензоров:", elem_tensor_preds.flatten())
print("Функция NumPy:", numpy_tensor_preds.flatten())

Сравнение на не обученной модели:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Модель: [0.5        0.41662967 0.2904222  0.31214076 0.39482468 0.47506458
 0.28091362 0.37751582]
Поэлементная функция тензоров: [0.5        0.24708189 0.3787405  0.2014875  0.25692576 0.1036882
 0.1941698  0.08168372]
Функция NumPy: [0.5        0.41662973 0.2904222  0.31214073 0.39482468 0.47506455
 0.28091362 0.3775158 ]


In [38]:
# Обучение модели
model.fit(X, y, epochs=100, verbose=0)
trained_weights = model.get_weights()

In [39]:
# Сравнение результатов на обученной модели
print("Сравнение на обученной модели:")
model_trained_preds = model.predict(X)
elem_tensor_preds = elem_tensor_function(X, trained_weights)
numpy_tensor_preds = numpy_tensor_function(X, trained_weights)
print("Модель:", model_trained_preds.flatten())
print("Поэлементная функция тензоров:", elem_tensor_preds.flatten())
print("Функция NumPy:", numpy_tensor_preds.flatten())

Сравнение на обученной модели:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Модель: [0.5246284  0.65190685 0.52259594 0.5919254  0.53981125 0.6469394
 0.53317946 0.58664453]
Поэлементная функция тензоров: [0.5246284  0.61104405 0.52547115 0.52351665 0.64366686 0.7144266
 0.559517   0.6375781 ]
Функция NumPy: [0.5246284  0.6519069  0.522596   0.5919254  0.5398112  0.64693934
 0.5331795  0.5866446 ]
